<a href="https://colab.research.google.com/github/MohamadElnomrossie/Omdena-seniment-analysis/blob/rest_ML_models(Naive_Bayes)/rest_ML_models(Naive_Bayes).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

In [5]:
df_train = pd.read_csv('train.csv')
df_valid = pd.read_csv('val.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2.0,True,:green_heart::green_heart::green_heart:,['https://t.co/Mzf90Ta5g1']
1,2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"['saud', 'Shubratanmyeh']",15.0,True,:sparkles:,['https://t.co/c2NXzNCdLU']
2,5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative,انا كامراه يوجعني كل حزن نساء العالم سواء تعنف...,['fatemaar7'],50.0,False,NaN,['https://t.co/c494qBVPx4']
3,7,@AliFCD @ShamsanM @AlyemenNor @space_dist @m2r...,negative,غسق وسبيس وحلوه الحلوات هم اشخاص لهم هويتهم وك...,"['AliFCD', 'ShamsanM', 'AlyemenNor', 'space', ...",32.0,False,NaN,[]
4,9,RT @Dresraamohammed: السؤال بقى للناس الي شغال...,negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,['Dresraamohammed'],24.0,True,:person_facepalming_light_skin_tone::female_si...,[]


In [23]:
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def apply_tf_idf_cnt_vec(train,test):
  union = FeatureUnion([("tf_idf_ngra_3", TfidfVectorizer(min_df=1,ngram_range=(1, 3),binary=True,max_features=700)),
                    ( "tf_idf_ngra_2", TfidfVectorizer(min_df=1,ngram_range=(1, 2),binary=True,max_features=700)),
                  ("tf_idf_ngra_1",TfidfVectorizer(min_df=1,ngram_range=(1, 1),binary=True,max_features=700)),
                  ("tf_idf_ngra_5", TfidfVectorizer(min_df=1,ngram_range=(1, 5),binary=True,max_features=700)),
                  ("tf_idf_ngra_7", TfidfVectorizer(min_df=1,ngram_range=(1, 7),binary=True,max_features=700)),
                  ("tf_idf_ngra_3_ch",TfidfVectorizer(min_df=1,ngram_range=(1, 3),binary=True,max_features=700,analyzer='char')),
                  ("tf_idf_ngra_2_ch",TfidfVectorizer(min_df=1,ngram_range=(1, 2),binary=True,max_features=700,analyzer='char')),
                  ("tf_idf_ngra_1_ch",TfidfVectorizer(min_df=1,ngram_range=(1, 1),binary=True,max_features=700,analyzer='char')),
                  ("tf_idf_ngra_5_ch",TfidfVectorizer(min_df=1,ngram_range=(1, 5),binary=True,max_features=700,analyzer='char')),
                  ("tf_idf_ngra_7_ch", TfidfVectorizer(min_df=1,ngram_range=(1, 7),binary=True,max_features=700,analyzer='char')),
                  ("cnt_vec_ngra_3", CountVectorizer(min_df=1,ngram_range=(1, 3),binary=True,max_features=700)),
                  ("cnt_vec_ngra_5", CountVectorizer(min_df=1,ngram_range=(1, 5),binary=True,max_features=700))
                                  
                                  
                                  ])
  train_feat_vec=union.fit_transform(train)
  test_feat_vec=union.transform(test)
  return train_feat_vec,test_feat_vec

df_train.dropna(inplace=True)
df_test.dropna(inplace=True)
train_vec,test_vec=apply_tf_idf_cnt_vec(df_train['cleaned_text'],df_test['cleaned_text'])

## Naive Bayes Model

In [25]:
clf = GaussianNB()
clf.fit(train_vec.toarray(),df_train['Class_camel'])

GaussianNB(priors=None, var_smoothing=1e-09)

In [26]:
from sklearn import metrics
y_pred = clf.predict(train_vec.toarray())
print(metrics.classification_report(df_train['Class_camel'], y_pred))

              precision    recall  f1-score   support

    negative       1.00      0.75      0.86       302
     neutral       0.74      1.00      0.85       103
    positive       0.81      0.93      0.87       257

    accuracy                           0.86       662
   macro avg       0.85      0.90      0.86       662
weighted avg       0.89      0.86      0.86       662



In [27]:
y_pred = clf.predict(test_vec.toarray())
print(metrics.classification_report(df_test['Class_camel'], y_pred))

              precision    recall  f1-score   support

    negative       0.62      0.55      0.58       173
     neutral       0.31      0.28      0.30        57
    positive       0.65      0.74      0.69       201

    accuracy                           0.60       431
   macro avg       0.53      0.52      0.53       431
weighted avg       0.60      0.60      0.60       431



## Naive Bayes 2

In [30]:
clf2 = BernoulliNB()
clf2.fit(train_vec,df_train['Class_camel'])
y_pred = clf2.predict(test_vec.toarray())
print(metrics.classification_report(df_test['Class_camel'], y_pred))

              precision    recall  f1-score   support

    negative       0.74      0.49      0.59       173
     neutral       0.25      0.68      0.36        57
    positive       0.73      0.57      0.64       201

    accuracy                           0.55       431
   macro avg       0.57      0.58      0.53       431
weighted avg       0.67      0.55      0.58       431



## Naive Bayes 3

In [31]:
clf3 = MultinomialNB()
clf3.fit(train_vec,df_train['Class_camel'])
y_pred = clf3.predict(test_vec.toarray())
print(metrics.classification_report(df_test['Class_camel'], y_pred))

              precision    recall  f1-score   support

    negative       0.71      0.66      0.69       173
     neutral       0.38      0.49      0.43        57
    positive       0.72      0.70      0.71       201

    accuracy                           0.66       431
   macro avg       0.60      0.62      0.61       431
weighted avg       0.67      0.66      0.66       431



In [37]:
pickle.dump(clf3, open('NB_model', 'wb'))

## Adaboost

In [36]:
clf4 = AdaBoostClassifier(n_estimators=200, random_state=0)
clf4.fit(train_vec,df_train['Class_camel'])
y_pred = clf4.predict(test_vec.toarray())
print(metrics.classification_report(df_test['Class_camel'], y_pred))

              precision    recall  f1-score   support

    negative       0.65      0.54      0.59       173
     neutral       0.53      0.30      0.38        57
    positive       0.61      0.78      0.69       201

    accuracy                           0.62       431
   macro avg       0.60      0.54      0.55       431
weighted avg       0.62      0.62      0.61       431

